In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
#Hey here is some added text.
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

This is text

so is this

In [ ]:
# but this chunk has code
# so everything that is not code has to be commented out
# and now this file has a name.